# Predict if an image has oilpalm or not for WiDS Datathon

In this notebook we will see the implementation of a deep learning model using PyTorch and transfer learning and we will use this model to predict if an image has oilpalm or not.

### 1. Import libraries

In [0]:
# Required libraries
import json
import zipfile
import os
import pandas as pd
from PIL import Image
import random

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch import optim
from torch.optim import lr_scheduler
import torchvision

### 2. Download the data from Kaggle

In [0]:
root_path = '/content'

In [0]:
!pip install kaggle
!mkdir ~/.kaggle
!touch '/root/.kaggle/kaggle.json'
######################################################################################
# api_token = {"username":"username","key":"TOKEN_HERE"}
api_token = {"username":"username","key":"TOKEN"}
######################################################################################


with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
!kaggle competitions download -c widsdatathon2019

traininglabels.csv: Skipping, found more recently modified local copy (use --force to force download)
leaderboard_holdout_data.zip: Skipping, found more recently modified local copy (use --force to force download)
leaderboard_test_data.zip: Skipping, found more recently modified local copy (use --force to force download)
train_images.zip: Skipping, found more recently modified local copy (use --force to force download)
SampleSubmission.csv: Skipping, found more recently modified local copy (use --force to force download)


### 3. Unzip and observe the files

In [0]:
for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [0]:
df_training = pd.read_csv(root_path+'/traininglabels.csv')
tr_nsamples = len(df_training)
print("Examples in the dataset: ", tr_nsamples)
df_training.head()

Examples in the dataset:  15244


,image_id,has_oilpalm,score
0,img_000002017.jpg,0,0.7895
1,img_000012017.jpg,0,1.0000
2,img_000022017.jpg,0,1.0000
3,img_000072017.jpg,0,1.0000
4,img_000082017.jpg,0,1.0000


We observe we have 15244 files for training

In [0]:
path_test_data = root_path+'/leaderboard_test_data/'
test_files = os.listdir(path_test_data)
print ("Test samples qty: ", len(test_files))

test_files[0:10]

Test samples qty:  4356


['img_052832018.jpg',
 'img_033872018.jpg',
 'img_004672018.jpg',
 'img_067372017.jpg',
 'img_012932018.jpg',
 'img_098252017.jpg',
 'img_050582017.jpg',
 'img_061902017.jpg',
 'img_079922018.jpg',
 'img_075272018.jpg']

We have to predict if this images have oilpalm or not. We have 4356 images

In [0]:
df_sample_sub = pd.read_csv(root_path+'/SampleSubmission.csv')
print("Sample rows", len(df_sample_sub))
df_sample_sub.head(10)

Sample rows 6534


,image_id,has_oilpalm
0,img_000012018.jpg,1
1,img_000022018.jpg,1
2,img_000032017.jpg,1
3,img_000042017.jpg,1
4,img_000052017.jpg,1
5,img_000062017.jpg,1
6,img_000062018.jpg,1
7,img_000122017.jpg,1
8,img_000132018.jpg,1
9,img_000142018.jpg,1


This is an example of how to submit the answers.

Now, we observe how many images we have from each class. We observe that there are more images that do not contain oilpalm.

In [0]:
df_training['has_oilpalm'].value_counts()

0    14302
1      942
Name: has_oilpalm, dtype: int64

Now, we divide the dataset for training and validation using 80% of the images from each class for training and 20% for validation.
We create new folders: Train and Valid and also separate the classes inside those folders.

In [0]:
has_oilpalm= df_training[df_training['has_oilpalm']==1]
no_oilpalm= df_training[df_training['has_oilpalm']==0]

In [0]:
random.seed(30)
has_oilpalm_train= has_oilpalm.sample(frac=0.8)
has_oilpalm_valid= has_oilpalm.drop(has_oilpalm_train.index)

no_oilpalm_train= no_oilpalm.sample(frac=0.8)
no_oilpalm_valid= no_oilpalm.drop(no_oilpalm_train.index)

train= pd.concat([has_oilpalm_train, no_oilpalm_train],axis=0)
valid= pd.concat([has_oilpalm_valid, no_oilpalm_valid],axis=0)

In [0]:
old_path = root_path+'/train_images'
train_new_path = root_path+'/train'
valid_new_path= root_path+'/valid'

os.makedirs(train_new_path, exist_ok=True)
os.makedirs(valid_new_path, exist_ok=True)

for index, row in train.iterrows():
  tr_class = row['has_oilpalm']
  tr_image = row['image_id']
  # Create the class folder if it doesn't exists yet.
  os.makedirs(train_new_path+'/'+str(tr_class), exist_ok=True)
  # Copy files
  os.system('cp '+old_path+'/'+tr_image+' '+train_new_path+'/'+str(tr_class)+'/'+tr_image)

for index, row in valid.iterrows():
  tr_class = row['has_oilpalm']
  tr_image = row['image_id']
  # Create the class folder if it doesn't exists yet.
  os.makedirs(valid_new_path+'/'+str(tr_class), exist_ok=True)
  # Copy files
  os.system('cp '+old_path+'/'+tr_image+' '+valid_new_path+'/'+str(tr_class)+'/'+tr_image)

### 4.Data transformation

In [0]:
# Train dataset transformations
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

# Validation dataset transformation
valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [0]:
train_dir = 'train'
valid_dir= 'valid'

# Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_dataset= datasets.ImageFolder(valid_dir, transform=valid_transforms)

# Using the image datasets and the trasnforms, define the dataloaders
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloaders = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=True)

### 5. Define the model using transfer learning

In [0]:
model = models.densenet169(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.torch/models/densenet169-b2777c0a.pth
57365526it [00:00, 66107871.06it/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

# Define the classifier part with two classes    
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1664, 100)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.4)),
                          ('fc2', nn.Linear(100, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [0]:
# Unfroze las two parts of the model
for name, child in model.features.named_children():
   if name in ['norm5']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False

conv0 is frozen
norm0 is frozen
relu0 is frozen
pool0 is frozen
denseblock1 is frozen
transition1 is frozen
denseblock2 is frozen
transition2 is frozen
denseblock3 is frozen
transition3 is frozen
denseblock4 is frozen
norm5 is unfrozen


In [0]:
# Define the cost function, optimizer and scheduler
criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0006, momentum=0.9)
#scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device);

### 6. Train the model

In [0]:
epochs = 5  # Number of iterations
steps = 0
running_loss = 0
print_every = 5
valid_loss_min = np.Inf
for epoch in range(epochs):
  for inputs, labels in train_dataloaders:
    steps += 1

    inputs, labels = inputs.to(device), labels.to(device)
    
    optimizer.zero_grad()
        
    logps = model.forward(inputs)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss
  print(f"Epoch {epoch+1}/{epochs}.. ")
  print(f"Train loss: {running_loss/len(train_dataloaders):.4f}")
  running_loss=0
    
  valid_loss = 0
  accuracy = 0
  model.eval()
  with torch.no_grad():
    for inputs, labels in valid_dataloaders:
      inputs, labels = inputs.to(device), labels.to(device)
      logps = model.forward(inputs)
      batch_loss = criterion(logps, labels)
      valid_loss += batch_loss.item()
        
      # Calculate accuracy
      ps = torch.exp(logps)
      top_p, top_class = ps.topk(1, dim=1)
      equals = top_class == labels.view(*top_class.shape)
      accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    
  print(f"Valid loss: {valid_loss/len(valid_dataloaders):.3f}.. "
       f"Valid accuracy: {accuracy/len(valid_dataloaders):.3f}")
    
  if valid_loss/len(valid_dataloaders)<valid_loss_min:
    valid_loss_min= valid_loss/len(valid_dataloaders)
    torch.save(model.state_dict(), 'model.pt')
    print('Minimum valid loss', valid_loss_min)
    running_loss = 0
    model.train()
    scheduler.step()

Epoch 1/5.. 
Train loss: 0.1118
Valid loss: 0.035.. Valid accuracy: 0.990
Minimum valid loss 0.03516116131019468
Epoch 2/5.. 
Train loss: 0.0571
Valid loss: 0.029.. Valid accuracy: 0.991
Minimum valid loss 0.029225828824564815
Epoch 3/5.. 
Train loss: 0.0519
Valid loss: 0.027.. Valid accuracy: 0.991
Minimum valid loss 0.027257732891788084
Epoch 4/5.. 
Train loss: 0.0502
Valid loss: 0.026.. Valid accuracy: 0.991
Minimum valid loss 0.026458260059977572
Epoch 5/5.. 
Train loss: 0.0492
Valid loss: 0.027.. Valid accuracy: 0.993


### 7. Save and load the best model

In [0]:
checkpoint = {'input_size': 224,
              'output_size': 2,
              'cnn': 'densenet169',
              'classifier': model.classifier,
              'optimizer' : optimizer.state_dict(),
              'state_dict': model.state_dict()}
torch.save(checkpoint, 'checkpoint.pth')

In [0]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath, map_location=lambda storage, loc: storage)
    model = models.densenet169(pretrained=False)
    
    classifier= checkpoint['classifier']
    model.classifier= classifier
    
    model.load_state_dict(checkpoint['state_dict'], strict= False)
    
    return model

In [0]:
model =load_checkpoint('checkpoint.pth')
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

### 8. Make predictions

In [0]:
# Test transforms
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# This function will help us make the predictions
def predict_class(img_path):
    # load the image and return the predicted breed
    img= Image.open(img_path)
    img= test_transforms(img)
    img= img.unsqueeze(0).cuda()
    output= model(img)
    ps= torch.exp(output)
    _,top_class= ps.topk(k=1,dim=1)
    return top_class

In [0]:
path_test_data = root_path+'/leaderboard_test_data/'
test_files = sorted(os.listdir(path_test_data))

path_holdout_data = root_path+'/leaderboard_holdout_data/'
holdout_files = sorted(os.listdir(path_holdout_data))

In [0]:
results_test= pd.DataFrame(index= range(len(test_files)),columns=['image_id','has_oilpalm'])
results_holdout=pd.DataFrame(index= range(len(holdout_files)),columns=['image_id','has_oilpalm'])

In [0]:
for index in range(len(test_files)):
  image= test_files[index]
  path= path_test_data+image
  predicted_class= predict_class(path).cpu().detach().numpy()[0][0]
  results_test.iloc[index,0]=image
  results_test.iloc[index,1]=predicted_class
  
for index in range(len(holdout_files)):
  image= holdout_files[index]
  path= path_holdout_data+image
  predicted_class= predict_class(path).cpu().detach().numpy()[0][0]
  results_holdout.iloc[index,0]=image
  results_holdout.iloc[index,1]=predicted_class

In [0]:
results_total= pd.concat([results_test, results_holdout], axis=0)

### 9. Save predictions to a csv file

In [0]:
results_total.to_csv('Results_oilpalm.csv',index=False)

In [149]:
# Save the jupyter notebook as html file
!jupyter nbconvert --to html oilpalm.ipynb

[NbConvertApp] Converting notebook oilpalm.ipynb to html
[NbConvertApp] Writing 423158 bytes to oilpalm.html
